In [1]:
import sklearn as sk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Import Data

In [2]:
train = pd.read_csv("../data/train.csv")
test = pd.read_csv("../data/test.csv")
display(train.head())
display(test.head())

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


### Combine Train and Test Data

In [3]:
all_data = pd.concat([train, test], sort=False)

### Add Last Name

In [4]:
all_data["Family_Name"] = all_data['Name'].str.split(',', 1, expand=True)[0]
# Group Title
display(pd.DataFrame(all_data.groupby('Family_Name')['PassengerId'].nunique()))

,PassengerId
Family_Name,
Abbing,1
Abbott,3
Abelseth,2
Abelson,2
Abrahamsson,1
...,...
de Mulder,1
de Pelsmaeker,1
del Carlo,2


### Extract Title from Name

In [5]:
# Grouping Title
new_title = {
    'Mr' : 'Mr','Ms' : 'Ms','Mrs' : 'Mrs','Rev' : 'officer','Sir' : 'royalty','theCountess' : 'royalty','Dona' : 'royalty','Capt' : 'officer','Col' : 'officer','Don' : 'royalty','Dr' : 'officer','Jonkheer' : 'royalty','Lady' : 'royalty','Major' : 'officer','Master' : 'kid','Miss' : 'Ms','Mlle' : 'Ms','Mme' : 'Mrs'
}

#Add Title
def add_title(df):
    df['title'] = df['Name'].apply(lambda x: x.split(",")[1])
    df['title'] = df['title'].apply(lambda x: x.split(".")[0])
    df.title = df.title.str.replace(' ', '')
    
add_title(all_data)

#
display(pd.DataFrame(all_data.groupby('title')['PassengerId'].nunique()))

,PassengerId
title,
Capt,1
Col,4
Don,1
Dona,1
Dr,8
Jonkheer,1
Lady,1
Major,2
Master,61


### Add Cabin Section

In [6]:
# Combining T with X due to similar behaviour
all_data["Cabin"] = all_data["Cabin"].apply(lambda x: "X" if pd.isnull(x) or x == "T" else x)
all_data["Cabin_Section"] = all_data['Cabin'].str[0]
display(pd.DataFrame(all_data.groupby('Cabin_Section')['PassengerId'].nunique()))

,PassengerId
Cabin_Section,
A,22
B,65
C,94
D,46
E,41
F,21
G,5
X,1015


### Binning Age and Fare

In [7]:
# Afer Null value
all_data["Fare_Bins"] = pd.qcut(all_data["Fare"], 10)
all_data["Age_Bins"] = pd.qcut(all_data["Age"], 16)

### Family Size

In [8]:
all_data["Family_Size"] = all_data["SibSp"] + all_data["Parch"] + 1

### Family Size Grouped

In [9]:
# Grouping Family Sizes
family_map = {
    1: 'Alone', 
    2: 'Small', 
    3: 'Small', 
    4: 'Small', 
    5: 'Medium', 
    6: 'Medium', 
    7: 'Large', 
    8: 'Large', 
    11: 'Large'
}
all_data['Family_Size_Grouped'] = all_data['Family_Size'].map(family_map)

### Grouping Tickets

In [10]:
display(pd.DataFrame(all_data.groupby('Ticket')['PassengerId'].nunique()))

,PassengerId
Ticket,
110152,3
110413,3
110465,2
110469,1
110489,1
...,...
W./C. 6608,5
W./C. 6609,1
W.E.P. 5734,2


In [11]:
all_data["Ticket_Group_Size"] = all_data.groupby('Ticket')['Ticket'].transform('count')

### Suvival Rate

In [12]:
# Add Survival Rates (Percentage) using Ticket, Cabin and Family Name
def add_surival_rates(x, survival_rate_df, feature_name, new_feature):
    feature_val = x[feature_name]
    # For Test Data to see if same Ticket is present in Train data
    # For Training Data update survival rate only if this ticket is present in Test Data
    if feature_val in list(survival_rate_df[feature_name]):
        x[new_feature] = np.float(survival_rate_df[survival_rate_df[feature_name] == feature_val]['Survived'])
    return x

def add_survival_rates(all_data):
    # Split Data
    x_train = all_data[0:891]
    x_test = all_data[891:]
    
    # Get Mean Age by Ticket, Cabin and Family Name
    ticket_survival_rate_df = pd.DataFrame(x_train.groupby(['Ticket'])[['Survived']].mean().reset_index())
    cabin_survival_rate_df = pd.DataFrame(x_train.groupby(['Cabin'])['Survived'].mean().reset_index())
    name_survival_rate_df = pd.DataFrame(x_train.groupby(['Family_Name'])['Survived'].mean().reset_index())
    
    # Get List of Ticket, Family name & Cabins in Test Data
    test_ticket_list = list(x_test['Ticket'])
    test_name_list = list(x_test['Family_Name'])
    test_cabin_list = list(x_test['Cabin'])
    
    feature_name = ['Ticket', 'Cabin', 'Family_Name']
    survival_feature_names = ['Ticket_Survival_Rate', 'Cabin_Survival_Rate', 'Name_Survival_Rate']
    test_lists = [test_ticket_list, test_cabin_list, test_name_list]
    grouped_dfs = [ticket_survival_rate_df, cabin_survival_rate_df, name_survival_rate_df]
    
    for i, j, k, l in zip(feature_name, survival_feature_names, test_lists, grouped_dfs):
        x_test[[i, j]] = x_test[[i, j]].apply(lambda x: add_surival_rates(x, l, i, j), axis = 1)
        # Update Survival Rate only if Ticket/Cabin/Family Name is present in test data
        x_train[[i, j]] = x_train[[i,j]].apply(lambda x: add_surival_rates(x, l, i , j) if x[i] in k else x, axis = 1)
    
    all_data = pd.concat([x_train, x_test], sort=False)
    return all_data

In [13]:
# Add Mean Survival Rates
all_data['Ticket_Survival_Rate'] = np.mean(train['Survived'])
all_data['Cabin_Survival_Rate'] = np.mean(train['Survived'])
all_data['Name_Survival_Rate'] = np.mean(train['Survived'])

# Update Survival Rates
all_data = add_survival_rates(all_data)

C:\Users\Aman\AppData\Local\Temp/ipykernel_6500/1847956414.py:7: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x[new_feature] = np.float(survival_rate_df[survival_rate_df[feature_name] == feature_val]['Survived'])
C:\Users\Aman\AppData\Local\Temp/ipykernel_6500/1847956414.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_test[[i, j]] = x_test[[i, j]].apply(lambda x: add_surival_rates(x, l, i, j), axis = 1)
C:\Users\Am

### Separating Training and Testing Data

In [16]:
train = all_data[0:891]
test = all_data[891:]